In [5]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from CandleStickPatterns import *
from datetime import datetime, timedelta

In [6]:
symbol = "INFY"

def get_historical_stock_price(symbol, to_date = datetime.today().strftime("%d-%m-%Y"), 
                                   from_date = (datetime.today() - timedelta(days = 367)).strftime("%d-%m-%Y")):
    df = equity_history(symbol, "EQ", from_date, to_date)
    df = df.drop(columns = ['_id','CH_SYMBOL','CH_SERIES','CH_MARKET_TYPE', 'CH_LAST_TRADED_PRICE',
                                    'CH_PREVIOUS_CLS_PRICE','CH_TOTAL_TRADES','TIMESTAMP','createdAt',
                                    'updatedAt','__v','VWAP','mTIMESTAMP','CA'])
    df.columns = ['high', 'low', 'open', 'close', 'volume', 'total_traded_value', 
                  'high_52_week', 'low_52_week', 'ISIN', 'date']
    return df.set_index('date').sort_index()

In [7]:
data = get_historical_stock_price(symbol)
data['candle_body_length'] = (data['close'] - data['open']).abs()
data['candle_body_ratio'] = (data['candle_body_length'] * 100 / (data['high'] - data['low']))
data['candle_length'] = data['high'] - data['low']
data['day_change_perc'] = data['candle_body_length'] * 100 / data['close'].shift(1)
data['candle_color'] = np.where(data['open'] > data['close'], "Red", "Green")
data['body_min'], data['body_max'] = data[['open', 'close']].min(axis = 1), data[['open', 'close']].max(axis = 1)
data['upper_wick_length'] = data['high'] - data[['close','open']].max(axis = 1)
data['lower_wick_length'] = data[['close','open']].min(axis = 1) - data['low']
data['prev_candle_body_ratio'] = data['candle_body_ratio'].shift(1)
data['prev_candle_color'] = data['candle_color'].shift(1)
data['prev_candle_body_length'] = data['candle_body_length'].shift(1)
data['prev_close'] = data['close'].shift(1)
data['prev_open'] = data['open'].shift(1)
data['prev_body_min'] = data['body_min'].shift(1)
data['prev_body_max'] = data['body_max'].shift(1)

In [8]:
data['marubozu'] = marubozu(data)
data['spinning_top'] = spinning_top_doji(data)
data['paper_umbrella'] = paper_umbrella(data)
data['shooting_star'] = shooting_star(data)
data['engulfing'] = engulfing(data)
data['harami'] = harami(data)
data['partial_engulf'] = partial_engulf(data)
data['gap_open'] = gap_opening(data)
data['star_pattern'] = star(data)

In [9]:
data.head()

,high,low,open,close,volume,total_traded_value,high_52_week,low_52_week,ISIN,candle_body_length,...,spinning_top,paper_umbrella,shooting_star,current_prev_cbr_ratio,prev_current_cbr_ratio,engulfing,harami,partial_engulf,gap_open,star_pattern
date,,,,,,,,,,,,,,,,,,,,,
2021-09-13,1701.50,1675.20,1683.00,1691.90,4662374,7.876437e+09,1757.0,948.3,INE009A01021,8.90,...,,False,False,NaN,NaN,,,,,
2021-09-14,1695.95,1678.60,1695.95,1686.15,4945167,8.349299e+09,1757.0,948.3,INE009A01021,9.80,...,,False,False,1.101124,0.908163,,,,Gap Up,
2021-09-15,1715.10,1690.20,1693.25,1711.45,4062344,6.926434e+09,1757.0,948.3,INE009A01021,18.20,...,,False,False,1.857143,0.538462,,,,,
2021-09-16,1719.25,1691.45,1715.20,1702.25,3246568,5.522818e+09,1757.0,948.3,INE009A01021,12.95,...,,False,False,0.711538,1.405405,,,Dark Cloud,Gap Up,
2021-09-17,1718.00,1684.45,1697.00,1691.30,7454864,1.264763e+10,1757.0,948.3,INE009A01021,5.70,...,,False,False,0.440154,2.271930,,,,Gap Down,


In [ ]:
# 14 Jan - 19 Jan